In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import os

2022-10-11 14:12:50.442807: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
model_path = os.getcwd() + '/sentiment_model_v1.h5'
print(model_path)
print(os.getenv('LD_LIBRARY_PATH'))

/home/juancm/ml-ops/sentiment_classification_albert/sentiment_model_v1.h5
/home/juancm/miniconda3/envs/mlops-dev/lib


# Build model and load weights

In [55]:
URL_PREPROCESSOR = "http://tfhub.dev/tensorflow/albert_en_preprocess/3"
preprocessor = hub.KerasLayer(URL_PREPROCESSOR, name='preprocessing')
URL = "https://tfhub.dev/tensorflow/albert_en_base/3"

def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    encoder_inputs = preprocessor(text_input)
    encoder = hub.KerasLayer(URL, trainable=False, name='ALBERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    dropout_layer = tf.keras.layers.Dropout(0.1, name='dropout')
    # net = dropout_layer(net)
    net = tf.keras.layers.Dense(1, activation=tf.sigmoid, name='classifier')(net) # Add Sigmoid as the activation function to get binary result
    return tf.keras.Model(inputs=text_input, outputs=net, name='albert_model')

model = build_classifier_model()

In [56]:
# Evaluate the model with some random inputs

model.load_weights(model_path)

bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
print(model.predict(tf.constant(['bad'])))
print(model.predict(tf.constant(['this is nice!'])))
print(model.predict(tf.constant(['I do not feel good'])))
print(model.predict(tf.constant(['this is a terrible movie'])))

1/1 [==============================] - 0s 436ms/step
[[0.47539645]]
1/1 [==============================] - 0s 79ms/step
[[0.74396527]]
1/1 [==============================] - 0s 78ms/step
[[0.22135879]]
1/1 [==============================] - 0s 77ms/step
[[0.45387965]]


In [57]:
model.save("sentiment_model.h5", include_optimizer=False)

In [61]:
model = tf.keras.models.load_model("sentiment_model.h5",
                                    custom_objects={'KerasLayer':hub.KerasLayer})


In [63]:
# Convert h5 model to SavedModel so that it can be served

export_path = './saved_models/1/'
tf.saved_model.save(model, export_path)

INFO:tensorflow:Assets written to: ./saved_models/1/assets


INFO:tensorflow:Assets written to: ./saved_models/1/assets


In [62]:
print(model.predict(tf.constant(['this is a terrible movie'])))

1/1 [==============================] - 0s 474ms/step
[[0.45387965]]
